In [13]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import tensorflow as tf

In [ ]:
import opendatasets as od
import pandas
#levshprygin
#64bde7f60d56248a6c006bd765676724
od.download("https://www.kaggle.com/competitions/cnn-competitions/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: levshprygin
Your Kaggle Key: ··········


100%|██████████| 209M/209M [00:02<00:00, 74.5MB/s]



Extracting archive ./cnn-competitions/cnn-competitions.zip to ./cnn-competitions


In [ ]:
tf.device('/gpu:0')

In [ ]:
from tensorflow import keras

In [6]:
#WORKS
size = 299

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.15
                                                                  )    

train_ds = image_generator.flow_from_directory(
    directory='/content/cnn-competitions/train/train',
    class_mode='categorical',
    subset="training",
    batch_size=32,
    shuffle = True,
    target_size=(size, size))

validation_ds = image_generator.flow_from_directory(
    directory='/content/cnn-competitions/train/train',
    class_mode='categorical',
    subset="validation",
    batch_size=32,
    target_size=(size, size))


preprocess_input = keras.applications.inception_resnet_v2.preprocess_input

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

base_model = tf.keras.applications.InceptionResNetV2(input_shape=(size,size,3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

global_average_layer = keras.layers.GlobalAveragePooling2D()

dense_layer = tf.keras.layers.Dense(200)

prediction_layer =  tf.keras.layers.Softmax()


inputs = tf.keras.Input(shape=(size, size, 3))
x = data_augmentation(inputs)
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = dense_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

initial_epochs = 3

history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=validation_ds)

Found 85000 images belonging to 200 classes.
Found 15000 images belonging to 200 classes.
219070464/219055592 [==============================] - 1s 0us/step
Epoch 1/3
2657/2657 [==============================] - 1104s 408ms/step - loss: 1.3847 - accuracy: 0.6856 - val_loss: 1.1053 - val_accuracy: 0.7291
Epoch 2/3
2657/2657 [==============================] - 1081s 407ms/step - loss: 1.0247 - accuracy: 0.7466 - val_loss: 1.1096 - val_accuracy: 0.7317
Epoch 3/3
2657/2657 [==============================] - 1081s 407ms/step - loss: 0.9459 - accuracy: 0.7611 - val_loss: 1.0756 - val_accuracy: 0.7432


In [7]:
base_model.trainable = True

In [8]:
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  780


In [9]:
fine_tune_at = 700

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])


In [11]:
len(model.trainable_variables)

52

In [15]:
history_fine = model.fit(train_ds,
                         epochs=initial_epochs+2,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_ds)

Epoch 3/5
2657/2657 [==============================] - 1203s 453ms/step - loss: 0.6263 - accuracy: 0.8288 - val_loss: 0.8789 - val_accuracy: 0.7719
Epoch 4/5
2657/2657 [==============================] - 1196s 450ms/step - loss: 0.4070 - accuracy: 0.8828 - val_loss: 0.9224 - val_accuracy: 0.7727
Epoch 5/5
  69/2657 [..............................] - ETA: 16:48 - loss: 0.2006 - accuracy: 0.9393

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/drive/MyDrive/my_checkpoint_resinc_jesus')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/my_checkpoint_resinc_jesus')

In [16]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.15                                                                  
                                                                  )    
validation_ds = image_generator.flow_from_directory(
    directory='/content/cnn-competitions/train/train',
    class_mode='categorical',
    subset="validation",
    batch_size=32,
    target_size=(size, size))

loss, accuracy = model.evaluate(validation_ds)
print('Test accuracy :', accuracy)

Found 15000 images belonging to 200 classes.
469/469 [==============================] - 163s 348ms/step - loss: 0.9873 - accuracy: 0.7721
Test accuracy : 0.7720666527748108


In [ ]:
path = '/content/cnn-competitions/train/train/class_031/00161.jpg'
model.trainable = False
np.set_printoptions(suppress=True)
size = 299
img = tf.keras.preprocessing.image.load_img(
  path, grayscale=False, color_mode="rgb", target_size=(size,size)
)
img = np.expand_dims(img, axis=0)

# print(model.predict(input_arr)[0])
print(np.argmax(model.predict(img)[0]))

31


In [ ]:
import glob
import numpy as np
import pandas as pd

output = pd.DataFrame(columns=['ID','Lable'] )
model.trainable = False

files = glob.glob('/content/cnn-competitions/val_kaggle/val_kaggle/*')
files.sort()

c=1

for path in files:
  img = tf.keras.preprocessing.image.load_img(
    path, grayscale=False, color_mode="rgb", target_size=(size,size)
  )
  img = np.expand_dims(img, axis=0)
  # input_arr = preprocess_input(img)
  # input_arr = np.array([img])
  # print(np.argmax(model.predict(input_arr)[0]))
  
  output = output.append({'ID': path[48:-4], 'Lable': np.argmax(model.predict(img)[0])}, ignore_index=True)
  c+=1
output
output.to_csv('/content/drive/MyDrive/output_222.csv', index = False)